In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

class PolynomialRegression:
    def __init__(self, degree=2, method='gradient_descent', lr=0.001, iterations=10000):
        self.degree = degree
        self.method = method
        self.lr = lr
        self.iterations = iterations
        self.weights = None

    def create_polynomial_features(self, X):
        poly_features = [np.ones(X.shape[0])]  # Sabit terim (bias)
        for d in range(1, self.degree + 1):
            poly_features.append(X ** d)
        return np.column_stack(poly_features)

    def gradient_descent(self, X, y):
        theta = np.random.randn(X.shape[1])
        m = X.shape[0]

        for iteration in range(self.iterations):
            predictions = np.dot(X, theta)
            errors = predictions - y
            gradient = np.dot(X.T, errors) / m
            theta_new = theta - self.lr * gradient
            
            if np.linalg.norm(theta_new - theta) < 1e-6:
                print(f"Converged at iteration {iteration}")
                break
            
            theta = theta_new
        return theta
    
    def fit(self, X, y):
        X_poly = self.create_polynomial_features(X)
    
        if self.method == 'gradient_descent':
            self.weights = self.gradient_descent(X_poly, y)
        else:
            raise ValueError("'method' should be 'gradient_descent'")

    def predict(self, X):
        X_poly = self.create_polynomial_features(X)
        return np.dot(X_poly, self.weights)

# Verileri yükleme ve hazırlama
data = pd.read_csv('processed_dataset.csv')

# Kategorik değişkeni sayısal değere dönüştürme
le = LabelEncoder()
data['city'] = le.fit_transform(data['city'])

# Aykırı değerleri işleme
def handle_outliers_iqr_all_columns(data):
    for column_name in data.columns:
        Q1 = data[column_name].quantile(0.25)  # Alt çeyrek (25. yüzdelik)
        Q3 = data[column_name].quantile(0.75)  # Üst çeyrek (75. yüzdelik)
        IQR = Q3 - Q1  # IQR hesaplama

        lower_bound = Q1 - 1.5 * IQR  # Alt sınır
        upper_bound = Q3 + 1.5 * IQR  # Üst sınır

        # Alt sınırın altındaki ve üst sınırın üstündeki aykırı değerleri çeyrekler arası aralığa getirme
        data[column_name] = data[column_name].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    
    return data

data = handle_outliers_iqr_all_columns(data)

# Özellik ve hedef değişkenleri ayırma
X = data.drop(columns=['opened_inside_year','carried_over_from','finished','carried_over_to']).values
y = data['opened_inside_year'].values

# Verileri normalleştirme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Veriyi eğitim ve test setlerine ayırma
# İlk 81 satır 2021 yılına aittir
X_train, X_test, y_train, y_test = X_scaled[81:], X_scaled[:81], y[81:], y[:81]

# Gradient Descent ile model oluşturma ve eğitme
model_gd = PolynomialRegression(degree=2, method='gradient_descent', lr=0.1, iterations=1000)
model_gd.fit(X_train, y_train)

# Tahminler
y_pred_gd = model_gd.predict(X_test)

# İlk 10 satır için tahmin edilen ve gerçek y değerlerini yazdırma (gradient descent)
print("Gradient Descent Predictions:")
for i in range(10):
    print("Predicted:", y_pred_gd[i], "\tReal:", y_test[i])

# Her tahmin için mutlak yüzde hatasını hesaplama (gradient descent)
absolute_percentage_errors_gd = np.abs((y_test - y_pred_gd) / y_test)

# Ortalama mutlak yüzde hatasını hesaplama (gradient descent)
mape_gd = np.mean(absolute_percentage_errors_gd)

# MAPE'yi doğruluğa dönüştürme (gradient descent)
accuracy_gd = 1 - mape_gd

# Doğruluğu yüzdeye dönüştürme (gradient descent)
percentage_accuracy_gd = accuracy_gd * 100

print("Average Percentage Accuracy (Gradient Descent):", percentage_accuracy_gd)


Gradient Descent Predictions:
Predicted: 100082.58562952575 	Real: 103840.0
Predicted: 64946.493421606996 	Real: 67719.0
Predicted: 25309.654981168278 	Real: 25475.0
Predicted: 17256.140438355655 	Real: 18984.0
Predicted: 71315.91449542914 	Real: 73542.0
Predicted: 31055.363723227652 	Real: 33447.0
Predicted: 100099.88433619076 	Real: 103840.0
Predicted: 87188.96141311279 	Real: 87538.0
Predicted: 55949.956460654525 	Real: 59477.0
Predicted: 72519.6888824062 	Real: 83130.0
Average Percentage Accuracy (Gradient Descent): 91.60710178816727


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

class PolynomialRegression:
    def __init__(self, degree=2, method='normal_equation', lr=0.001, iterations=10000):
        self.degree = degree
        self.method = method
        self.lr = lr
        self.iterations = iterations
        self.weights = None

    def create_polynomial_features(self, X):
        poly_features = [np.ones(X.shape[0])]  # Sabit terim (bias)
        for d in range(1, self.degree + 1):
            poly_features.append(X ** d)
        return np.column_stack(poly_features)

    def gradient_descent(self, X, y):
        theta = np.random.randn(X.shape[1])
        m = X.shape[0]

        for iteration in range(self.iterations):
            predictions = np.dot(X, theta)
            errors = predictions - y
            gradient = np.dot(X.T, errors) / m
            theta_new = theta - self.lr * gradient
            
            if np.linalg.norm(theta_new - theta) < 1e-6:
                print(f"Converged at iteration {iteration}")
                break
            
            theta = theta_new
        return theta

     def fit(self, X, y):
        X_poly = self.create_polynomial_features(X)
    
        if self.method == 'gradient_descent':
            self.weights = self.gradient_descent(X_poly, y)
        else:
            raise ValueError("'method' should be 'gradient_descent'")

    def predict(self, X):
        X_poly = self.create_polynomial_features(X)
        return np.dot(X_poly, self.weights)

# Verileri yükleme ve hazırlama
data = pd.read_csv('processed_dataset.csv')

# Kategorik değişkeni sayısal değere dönüştürme
le = LabelEncoder()
data['city'] = le.fit_transform(data['city'])

# Aykırı değerleri işleme
def handle_outliers_iqr_all_columns(data):
    for column_name in data.columns:
        Q1 = data[column_name].quantile(0.25)  # Alt çeyrek (25. yüzdelik)
        Q3 = data[column_name].quantile(0.75)  # Üst çeyrek (75. yüzdelik)
        IQR = Q3 - Q1  # IQR hesaplama

        lower_bound = Q1 - 1.5 * IQR  # Alt sınır
        upper_bound = Q3 + 1.5 * IQR  # Üst sınır

        # Alt sınırın altındaki ve üst sınırın üstündeki aykırı değerleri çeyrekler arası aralığa getirme
        data[column_name] = data[column_name].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    
    return data

data = handle_outliers_iqr_all_columns(data)

# Özellik ve hedef değişkenleri ayırma
X = data.drop(columns=['opened_inside_year','carried_over_from','finished','carried_over_to']).values
y = data['opened_inside_year'].values

# Verileri normalleştirme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA uygulama
pca = PCA(n_components=4)  # Kaç bileşen kullanılacak
X_pca = pca.fit_transform(X_scaled)

# PCA sonrası bileşenlerin dağılımını kontrol etme
print("PCA sonrası bileşenlerin dağılımı:")
print(X_pca[:10])  # İlk 10 örneğin bileşen değerlerini yazdırma

# Veriyi eğitim ve test setlerine ayırma
# İlk 81 satır 2021 yılına aittir
X_train, X_test, y_train, y_test = X_pca[81:], X_pca[:81], y[81:], y[:81]

# Gradient Descent ile model oluşturma ve eğitme
# Öğrenme oranını daha küçük bir değere ayarlayarak deneyin
model_gd = PolynomialRegression(degree=2, method='gradient_descent', lr=0.01, iterations=1000)
model_gd.fit(X_train, y_train)

# Tahminler
y_pred_gd = model_gd.predict(X_test)

# İlk 10 satır için tahmin edilen ve gerçek y değerlerini yazdırma (gradient descent)
print("Gradient Descent Predictions:")
for i in range(10):
    print("Predicted:", y_pred_gd[i], "\tReal:", y_test[i])

# Her tahmin için mutlak yüzde hatasını hesaplama (gradient descent)
absolute_percentage_errors_gd = np.abs((y_test - y_pred_gd) / y_test)

# Ortalama mutlak yüzde hatasını hesaplama (gradient descent)
mape_gd = np.mean(absolute_percentage_errors_gd)

# MAPE'yi doğruluğa dönüştürme (gradient descent)
accuracy_gd = 1 - mape_gd

# Doğruluğu yüzdeye dönüştürme (gradient descent)
percentage_accuracy_gd = accuracy_gd * 100

print("Average Percentage Accuracy (Gradient Descent):", percentage_accuracy_gd)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 38)

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA

class PolynomialRegression:
    def __init__(self, degree=2, method='gradient_descent', lr=0.001, iterations=10000):
        self.degree = degree
        self.method = method
        self.lr = lr
        self.iterations = iterations
        self.weights = None

    def create_polynomial_features(self, X):
        poly_features = [np.ones(X.shape[0])]  # Sabit terim (bias)
        for d in range(1, self.degree + 1):
            poly_features.append(X ** d)
        return np.column_stack(poly_features)

    def gradient_descent(self, X, y):
        theta = np.random.randn(X.shape[1])
        m = X.shape[0]

        for iteration in range(self.iterations):
            predictions = np.dot(X, theta)
            errors = predictions - y
            gradient = np.dot(X.T, errors) / m
            theta_new = theta - self.lr * gradient
            
            if np.linalg.norm(theta_new - theta) < 1e-6:
                print(f"Converged at iteration {iteration}")
                break
            
            theta = theta_new
        return theta

    def fit(self, X, y):
        X_poly = self.create_polynomial_features(X)
        if self.method == 'normal_equation':
            self.weights = self.normal_equation(X_poly, y)
        elif self.method == 'gradient_descent':
            self.weights = self.gradient_descent(X_poly, y)
        else:
            raise ValueError("Method must be 'normal_equation' or 'gradient_descent'")

    def predict(self, X):
        X_poly = self.create_polynomial_features(X)
        return np.dot(X_poly, self.weights)

# Verileri yükleme ve hazırlama
data = pd.read_csv('processed_dataset.csv')

# Kategorik değişkeni sayısal değere dönüştürme
le = LabelEncoder()
data['city'] = le.fit_transform(data['city'])

# Aykırı değerleri işleme
def handle_outliers_iqr_all_columns(data):
    for column_name in data.columns:
        Q1 = data[column_name].quantile(0.25)  # Alt çeyrek (25. yüzdelik)
        Q3 = data[column_name].quantile(0.75)  # Üst çeyrek (75. yüzdelik)
        IQR = Q3 - Q1  # IQR hesaplama

        lower_bound = Q1 - 1.5 * IQR  # Alt sınır
        upper_bound = Q3 + 1.5 * IQR  # Üst sınır

        # Alt sınırın altındaki ve üst sınırın üstündeki aykırı değerleri çeyrekler arası aralığa getirme
        data[column_name] = data[column_name].apply(lambda x: lower_bound if x < lower_bound else (upper_bound if x > upper_bound else x))
    
    return data

data = handle_outliers_iqr_all_columns(data)

# Özellik ve hedef değişkenleri ayırma
X = data.drop(columns=['opened_inside_year','carried_over_from','finished','carried_over_to']).values
y = data['opened_inside_year'].values

# Verileri normalleştirme
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA uygulama
pca = PCA(n_components=5)  # Kaç bileşen kullanılacak
X_pca = pca.fit_transform(X_scaled)

# PCA sonrası bileşenlerin dağılımını kontrol etme
print("PCA sonrası bileşenlerin dağılımı:")
print(X_pca[:10])  # İlk 10 örneğin bileşen değerlerini yazdırma

# Veriyi eğitim ve test setlerine ayırma
# İlk 81 satır 2021 yılına aittir
X_train, X_test, y_train, y_test = X_pca[81:], X_pca[:81], y[81:], y[:81]

# Gradient Descent ile model oluşturma ve eğitme
# Öğrenme oranını daha küçük bir değere ayarlandı
model_gd = PolynomialRegression(degree=2, method='gradient_descent', lr=0.01, iterations=1000)
model_gd.fit(X_train, y_train)

# Tahminler
y_pred_gd = model_gd.predict(X_test)

# İlk 10 satır için tahmin edilen ve gerçek y değerlerini yazdırma (gradient descent)
print("Gradient Descent Predictions:")
for i in range(10):
    print("Predicted:", y_pred_gd[i], "\tReal:", y_test[i])

# Her tahmin için mutlak yüzde hatasını hesaplama (gradient descent)
absolute_percentage_errors_gd = np.abs((y_test - y_pred_gd) / y_test)

# Ortalama mutlak yüzde hatasını hesaplama (gradient descent)
mape_gd = np.mean(absolute_percentage_errors_gd)

# MAPE'yi doğruluğa dönüştürme (gradient descent)
accuracy_gd = 1 - mape_gd

# Doğruluğu yüzdeye dönüştürme (gradient descent)
percentage_accuracy_gd = accuracy_gd * 100

print("Average Percentage Accuracy (Gradient Descent):", percentage_accuracy_gd)


PCA sonrası bileşenlerin dağılımı:
[[ 4.04503133  0.8438743  -1.7561251   0.03569797 -0.08655409]
 [ 2.11554961  0.42507407 -1.75895007  0.60491666  0.34246759]
 [-0.44220761 -1.2927094  -1.22862457  0.2363895   0.11023636]
 [-0.74799604 -0.4485998  -1.64086241  0.06180665 -0.04454804]
 [ 1.48849157 -1.93268471 -0.81041489 -0.45815851  0.2775129 ]
 [-0.23658852  0.55337757 -2.1010855  -0.44992996  0.13275606]
 [ 4.046167    0.88182612 -1.77404967  0.02769094 -0.08505912]
 [ 2.27819023 -1.92677972 -0.75792197  0.14022227  0.91477641]
 [ 0.96522309 -1.48336472 -1.03509193 -0.2127999   0.12485653]
 [ 1.32670081 -0.36444288 -1.60780627 -0.77782868  0.70753209]]
Gradient Descent Predictions:
Predicted: 105598.79058640542 	Real: 103840.0
Predicted: 61446.563092625714 	Real: 67719.0
Predicted: 24651.137829764783 	Real: 25475.0
Predicted: 18494.23754085901 	Real: 18984.0
Predicted: 69650.25370095864 	Real: 73542.0
Predicted: 32644.40732769314 	Real: 33447.0
Predicted: 105780.25945460753 	Real: